In [ ]:
from glob import glob
import pandas as pd

files = glob("/Users/brett/Downloads/Tati's Amoebae Slack export Feb 24 2016 - Jan 5 2024/*/20*.json")
files[:10]

In [ ]:
users = pd.read_json("/Users/brett/Downloads/Tati's Amoebae Slack export Feb 24 2016 - Jan 5 2024/users.json").set_index("id").name
users

In [ ]:
df = pd.concat([pd.read_json(f).assign(channel=f.split("/")[-2]) for f in files])[['text', 'user', 'channel']].astype("string[pyarrow]")
df['user_channel'] = df['channel'].str.cat(df['user'], sep=';')
df['text'] += '\n'
df

In [ ]:
for id, name in users.items():
    df["text"] = df.text.str.replace(f"<@{id}>", f"@{name}")

In [ ]:
convos = df.groupby([(df['user_channel'] != df['user_channel'].shift(1)).astype("bool").cumsum(), "user", "channel"]).text.sum().reset_index()
convos

In [ ]:
import json
import numpy as np
from tqdm.auto import tqdm

target_user = "U0NSX3SCT"
prompt = "You are John Flinchbaugh talking to his friends on Slack. You are a Dallas Mavericks and Texas Rangers fan, you like music and have a very silly non-sensical Slack persona."
user_inds, = np.where(convos['user'].fillna("") == target_user)
with open("all_messages.jsonl", "w") as f:
    for i in tqdm(user_inds):
        json.dump({
            "messages": [
                {"role": "system", "content": prompt}, 
                {"role": "user", "content": convos.text.iloc[i - 1] if convos.channel.iloc[i - 1] == convos.channel.iloc[i] else ""}, 
                {"role": "assistant", "content": convos.text.iloc[i]}
            ]
        }, f)
        f.write("\n")

In [ ]:

from openai import OpenAI
client = OpenAI()

message_file = client.files.create(
  file=open("all_messages.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
job = client.fine_tuning.jobs.create(
  training_file=message_file.id,
  model="gpt-3.5-turbo"
)

job